<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# Assignment 7: Output Analysis (Multiple Model)

Summer Semester 24


Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Agenda

* Output Analysis (Multiple Model)
* Metamodels


In [35]:
import simpy
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

## Car wash example

In [36]:
class EventLogger:
    def __init__(self):
        self.logs = []
        self.replication = None
        self.seed = None
        self.system_id = None #used to identify the system in the logs
    
    def set_system_id(self, system_id):
        self.system_id = system_id

    def set_replication_info(self, replication, seed):
        self.replication = replication
        self.seed = seed

    def log_car_arrival(self, entity, time, interarrival_time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_arrival', 'event_key': entity, #simulation info
                          'interarrival_time': interarrival_time
                          })
    
    def log_car_wash_request(self, entity, time, machine_id, processing_time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_wash_request', 'event_key': entity, #simulation info
                          'machine_id': machine_id, 'processing_time': processing_time
                          })

    def log_car_departure(self, entity, time):
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_departure', 'event_key': entity #simulation info
                          })

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

#### System 1

2 Machines with a common queue

In [37]:
class Carwash_S1:
    def __init__(self, env, logger, n_machines=2):
        self.env = env
        self.machine = simpy.Resource(env, n_machines)
        self.logger = logger

    def wash(self, processing_time):
        yield self.env.timeout(processing_time)

In [38]:
class Car_S1:
    def __init__(self, env, name, carwash, logger, interarrival_time, processing_time):
        self.env = env
        self.name = name
        self.logger = logger
        self.interarrival_time = interarrival_time
        self.processing_time = processing_time

        self.env.process(self.run(carwash))

    def run(self, carwash):
        
        # Log the arrival
        self.logger.log_car_arrival(self.name, self.env.now, self.interarrival_time)
        with carwash.machine.request() as request:
            yield request

            # Log the request for car wash
            self.logger.log_car_wash_request(self.name, self.env.now, 'unique_machine', self.processing_time)
            yield self.env.process(carwash.wash(self.processing_time))

            #Log the departure
            self.logger.log_car_departure(self.name, self.env.now)

#### System 2

- 2 Machines with a dedicated queue
- Entities pick the machine with shorter queue

In [39]:
class Carwash_S2:
    def __init__(self, env, logger):
        self.env = env
        self.machine1 = simpy.Resource(env, 1)
        self.machine2 = simpy.Resource(env, 1)
        self.logger = logger

    def wash(self, processing_time):
        yield self.env.timeout(processing_time)

In [40]:
class Car_S2:
    def __init__(self, env, name, carwash, logger, interarrival_time, processing_time):
        self.env = env
        self.name = name
        self.logger = logger
        self.interarrival_time = interarrival_time
        self.processing_time = processing_time

        self.env.process(self.run(carwash))

    def run(self, carwash):
        
        # Log the arrival
        self.logger.log_car_arrival(self.name, self.env.now, self.interarrival_time)

        # Check which machine has the shortest queue
        if len(carwash.machine1.queue) <= len(carwash.machine2.queue):
            with carwash.machine1.request() as request: 
                yield request

                # Log the request for car wash
                self.logger.log_car_wash_request(self.name, self.env.now, 'machine_1', self.processing_time)
                yield self.env.process(carwash.wash(self.processing_time))
        else:
            with carwash.machine2.request() as request:
                yield request

                # Log the request for car wash
                self.logger.log_car_wash_request(self.name, self.env.now, 'machine_2', self.processing_time)
                yield self.env.process(carwash.wash(self.processing_time))

        #Log the departure
        self.logger.log_car_departure(self.name, self.env.now)            

#### Entity generator

In [41]:
def car_generator(env, system, carwash, logger):
    '''
    system: system configuration
    '''
    car_count = 0
    
    # Create cars while the simulation is running
    while True:
        iat = np.random.exponential(1/0.5) #interarrival time
        pt = np.random.exponential(1/0.3) #processing time

        yield env.timeout(iat)

        if system == 1:
            Car_S1(env, f'Car {car_count}', carwash, logger, iat, pt)
        elif system == 2:
            Car_S2(env, f'Car {car_count}', carwash, logger, iat, pt)
        car_count += 1

## Run Simulation

In [42]:
# parameters
SIM_TIME = 8*60    # Simulation time in minutes
N_REPLICATIONS = 10 # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger = EventLogger()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)

    #set replication id and random seed
    logger.set_replication_info(i, seed)

    ######################################
    ############## SYSTEM 1 ##############
    ######################################

    #set system id
    logger.set_system_id(1)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s1 = Carwash_S1(env, logger)

    #define processes
    env.process(car_generator(env, 1, carwash_s1, logger))

    # Execute
    env.run(until=SIM_TIME)

    ######################################
    ############## SYSTEM 2 ##############
    ######################################
    
    #set system id
    logger.set_system_id(2)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s2 = Carwash_S2(env, logger)

    #define processes
    env.process(car_generator(env, 2, carwash_s2, logger))

    # Execute
    env.run(until=SIM_TIME)

print('... Done')

Running Simulation...
Running Replication 0 with seed: 72 ...
Running Replication 1 with seed: 10 ...
Running Replication 2 with seed: 62 ...
Running Replication 3 with seed: 44 ...
Running Replication 4 with seed: 76 ...
Running Replication 5 with seed: 93 ...
Running Replication 6 with seed: 41 ...
Running Replication 7 with seed: 65 ...
Running Replication 8 with seed: 23 ...
Running Replication 9 with seed: 53 ...
... Done


In [43]:
events_df = logger.get_logs_df()

events_df

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,72,0.225751,car_arrival,Car 0,0.225751,NaN,NaN
1,1,0,72,0.225751,car_wash_request,Car 0,NaN,unique_machine,3.843660
2,1,0,72,1.757026,car_arrival,Car 1,1.531275,NaN,NaN
3,1,0,72,1.757026,car_wash_request,Car 1,NaN,unique_machine,1.535820
4,1,0,72,2.821174,car_arrival,Car 2,1.064148,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14811,2,9,53,473.344728,car_departure,Car 232,NaN,NaN,NaN
14812,2,9,53,473.344728,car_wash_request,Car 234,NaN,machine_2,8.186629
14813,2,9,53,474.284445,car_departure,Car 238,NaN,NaN,NaN
14814,2,9,53,474.284445,car_wash_request,Car 239,NaN,machine_1,22.140490


In [44]:
boolean_mask = (events_df['replication_id'] == 0) & (events_df['event_key'].isin(['Car 0', 'Car 1']) )

events_df[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,72,0.225751,car_arrival,Car 0,0.225751,NaN,NaN
772,2,0,72,1.227962,car_arrival,Car 0,1.227962,NaN,NaN
7,1,0,72,4.069411,car_departure,Car 0,NaN,NaN,NaN
774,2,0,72,2.761386,car_departure,Car 0,NaN,NaN,NaN
1,1,0,72,0.225751,car_wash_request,Car 0,NaN,unique_machine,3.843660
773,2,0,72,1.227962,car_wash_request,Car 0,NaN,machine_1,1.533423
2,1,0,72,1.757026,car_arrival,Car 1,1.531275,NaN,NaN
775,2,0,72,6.392531,car_arrival,Car 1,5.164569,NaN,NaN
5,1,0,72,3.292846,car_departure,Car 1,NaN,NaN,NaN
777,2,0,72,8.271747,car_departure,Car 1,NaN,NaN,NaN


In [45]:
boolean_mask = (events_df['replication_id'] == 1) & (events_df['event_key'].isin(['Car 0', 'Car 1']) )

events_df[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
1424,1,1,10,2.950869,car_arrival,Car 0,2.950869,NaN,NaN
2199,2,1,10,3.138433,car_arrival,Car 0,3.138433,NaN,NaN
1426,1,1,10,3.020770,car_departure,Car 0,NaN,NaN,NaN
2201,2,1,10,5.524546,car_departure,Car 0,NaN,NaN,NaN
1425,1,1,10,2.950869,car_wash_request,Car 0,NaN,unique_machine,0.069901
2200,2,1,10,3.138433,car_wash_request,Car 0,NaN,machine_1,2.386113
1427,1,1,10,4.959191,car_arrival,Car 1,2.008323,NaN,NaN
2202,2,1,10,6.242771,car_arrival,Car 1,3.104338,NaN,NaN
1437,1,1,10,9.564262,car_departure,Car 1,NaN,NaN,NaN
2211,2,1,10,7.601646,car_departure,Car 1,NaN,NaN,NaN


# Exercise / Tasks

**Tasks are independently of each other.**

## Task 1: Comparison of Two System Designs (CRN)

We'd like to compare the **average waiting time** of System 1 and System 2 but with Common Random Numbers (CRN). There are several ways of implementing this but today we'll guide you with one of them.

When implementing CRN, is important to note the following:

- It is never enough to simply use the same seed on the random-number generator(s). 
- Each random number used in one model for some purpose should be used for the same purpose in the second model—that is, the use of the random numbers must be synchronized. 
- For example, if the $i$ random number is used to generate the processing time of a machine for the 5th car in model 1, then the $i$ random number should be used for the very same purpose in model 2. 

-----

**Task 1.1: Create a stream of random numbers for the interarrival times and processing times. Use the same distributions as before, i.e. $exp(1/0.5)$ and $exp(1/0.3)$, respectively. The size of the stream should be at least 1000. Note that this stream must change whenever we set a new seed but must remain the same across system designs**

**Task 1.2: Modify the `car_generator_crn` to receive this stream of numbers (both, interarrival and processing times) and use them according the counter for each car. Modify also the parts of your code where you're calling this function.**

After these first 2 steps. You should have succesfully implemented CRN.

**Task 1.3: Create the "replication table". That is: a table that contains for every replication (<u>and now, for every system design</u>) the average waiting time within replications. This should be very similar to Task 1 of Assignment 6 but considering that now we have two different system designs.**

**Task 1.4: Evaluate whether the difference of average waiting times between the two system designs is statistically significant. You can use a CI or a hypothesis test for this purpose. Recall that when using CRN, we calculate the “synchronized” performance difference for each replication and then we conduct the CI or hypothesis test. Check slide 23 of the Lecture "Output Analysis of Multiple Models" for more details.**

-----

In [46]:
#your code here: modify the function to receive the streams as inputs
def car_generator_crn(env, system, carwash, logger, interarrival_stream, processing_stream):
    car_count = 0

    while True:
        iat = interarrival_stream[car_count]
        pt = processing_stream[car_count]

        yield env.timeout(iat)  # common random numbers

        if system == 1:
            Car_S1(env, f'Car {car_count}', carwash, logger, iat, pt)
        elif system == 2:
            Car_S2(env, f'Car {car_count}', carwash, logger, iat, pt)
        car_count += 1


In [47]:
# parameters
SIM_TIME = 8*60    # Simulation time in minutes
N_REPLICATIONS = 10 # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger_t1 = EventLogger()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)

    #generate streams of inter-arrival times and processing times ("common random numbers")
    stream_iat = np.random.exponential((1/0.5), 1000) #interarrival time
    stream_pt = np.random.exponential((1/0.3), 1000) #processing time

    #set replication id and random seed
    logger_t1.set_replication_info(i, seed)

    ######################################
    ############## SYSTEM 1 ##############
    ######################################

    #set system id
    logger_t1.set_system_id(1)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s1 = Carwash_S1(env, logger_t1)

    #define processes
    #your code here: pass the streams of inter-arrival times and processing times to the car generator function
    env.process(car_generator_crn(env, 1, carwash_s1, logger_t1, stream_iat, stream_pt))

    # Execute
    env.run(until=SIM_TIME)

    ######################################
    ############## SYSTEM 2 ##############
    ######################################

    #set system id
    logger_t1.set_system_id(2)

    # Create an environment and start the setup process
    env = simpy.Environment()

    #define resources
    carwash_s2 = Carwash_S2(env, logger_t1)

    #define processes
    #your code here: pass the streams of inter-arrival times and processing times to the car generator function
    env.process(car_generator_crn(env, 2, carwash_s2, logger_t1, stream_iat, stream_pt))

    # Execute
    env.run(until=SIM_TIME)

print('... Done')

Running Simulation...
Running Replication 0 with seed: 76 ...
Running Replication 1 with seed: 50 ...
Running Replication 2 with seed: 41 ...
Running Replication 3 with seed: 72 ...
Running Replication 4 with seed: 74 ...
Running Replication 5 with seed: 24 ...
Running Replication 6 with seed: 46 ...
Running Replication 7 with seed: 91 ...
Running Replication 8 with seed: 63 ...
Running Replication 9 with seed: 53 ...
... Done


In [48]:
#Leave this code unchanged
events_df_t1 = logger_t1.get_logs_df()

events_df_t1

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,76,0.744534,car_arrival,Car 0,0.744534,NaN,NaN
1,1,0,76,0.744534,car_wash_request,Car 0,NaN,unique_machine,1.504023
2,1,0,76,2.248557,car_departure,Car 0,NaN,NaN,NaN
3,1,0,76,4.162909,car_arrival,Car 1,3.418375,NaN,NaN
4,1,0,76,4.162909,car_wash_request,Car 1,NaN,unique_machine,0.273915
...,...,...,...,...,...,...,...,...,...
14448,2,9,53,476.533652,car_arrival,Car 239,1.778477,NaN,NaN
14449,2,9,53,476.533652,car_wash_request,Car 239,NaN,machine_2,1.571777
14450,2,9,53,476.714549,car_departure,Car 237,NaN,NaN,NaN
14451,2,9,53,476.714549,car_wash_request,Car 238,NaN,machine_1,6.022464


In [49]:
#Leave this code unchanged (check that the interarrival times and processing times are the same for the same car in both systems)
boolean_mask = (events_df_t1['replication_id'] == 0) & (events_df_t1['event_key'].isin(['Car 0', 'Car 1']) )

events_df_t1[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
0,1,0,76,0.744534,car_arrival,Car 0,0.744534,NaN,NaN
719,2,0,76,0.744534,car_arrival,Car 0,0.744534,NaN,NaN
2,1,0,76,2.248557,car_departure,Car 0,NaN,NaN,NaN
721,2,0,76,2.248557,car_departure,Car 0,NaN,NaN,NaN
1,1,0,76,0.744534,car_wash_request,Car 0,NaN,unique_machine,1.504023
720,2,0,76,0.744534,car_wash_request,Car 0,NaN,machine_1,1.504023
3,1,0,76,4.162909,car_arrival,Car 1,3.418375,NaN,NaN
722,2,0,76,4.162909,car_arrival,Car 1,3.418375,NaN,NaN
5,1,0,76,4.436825,car_departure,Car 1,NaN,NaN,NaN
724,2,0,76,4.436825,car_departure,Car 1,NaN,NaN,NaN


In [50]:
#Leave this code unchanged (check that the interarrival times and processing times are the same for the same car in both systems)
boolean_mask = (events_df_t1['replication_id'] == 1) & (events_df_t1['event_key'].isin(['Car 0', 'Car 1']) )

events_df_t1[boolean_mask].sort_values(by=['event_key', 'event_name', 'system_id'])

,system_id,replication_id,seed,event_time,event_name,event_key,interarrival_time,machine_id,processing_time
1438,1,1,50,1.364817,car_arrival,Car 0,1.364817,NaN,NaN
2160,2,1,50,1.364817,car_arrival,Car 0,1.364817,NaN,NaN
1452,1,1,50,8.445290,car_departure,Car 0,NaN,NaN,NaN
2172,2,1,50,8.445290,car_departure,Car 0,NaN,NaN,NaN
1439,1,1,50,1.364817,car_wash_request,Car 0,NaN,unique_machine,7.080473
2161,2,1,50,1.364817,car_wash_request,Car 0,NaN,machine_1,7.080473
1440,1,1,50,1.882573,car_arrival,Car 1,0.517757,NaN,NaN
2162,2,1,50,1.882573,car_arrival,Car 1,0.517757,NaN,NaN
1445,1,1,50,5.402245,car_departure,Car 1,NaN,NaN,NaN
2174,2,1,50,11.964961,car_departure,Car 1,NaN,NaN,NaN


In [61]:
#your code here: compute the waiting time for each car in each system and each replication (you may want to use pd.pivot_table for this)

waiting_time_df = pd.pivot_table(events_df, index = ["event_key", "system_id", "replication_id"], columns= "event_name", values="event_time")
waiting_time_df.reset_index(inplace=True)

waiting_time_df['waiting_time'] = waiting_time_df['car_departure'] - waiting_time_df['car_arrival']

waiting_time_df


event_name,event_key,system_id,replication_id,car_arrival,car_departure,car_wash_request,waiting_time
0,Car 0,1,0,0.225751,4.069411,0.225751,3.843660
1,Car 0,1,1,2.950869,3.020770,2.950869,0.069901
2,Car 0,1,2,0.068675,2.307329,0.068675,2.238654
3,Car 0,1,3,3.601707,3.970720,3.601707,0.369013
4,Car 0,1,4,0.744534,6.441826,0.744534,5.697292
...,...,...,...,...,...,...,...
5010,Car 99,2,5,234.138420,234.490330,234.138420,0.351910
5011,Car 99,2,6,194.078937,204.432789,197.975474,10.353852
5012,Car 99,2,7,168.013085,185.034977,179.604713,17.021892
5013,Car 99,2,8,176.042396,180.830808,176.042396,4.788412


In [115]:
#your code here: compute the average waiting time for each system and each replication (average waiting time within replications, for the two systems)

average_df = waiting_time_df.groupby(["system_id", "replication_id"]).agg(average_time=pd.NamedAgg(column="waiting_time",aggfunc="mean"))

average_df

average_time
system_id replication_id              
1         0                  15.918033
          1                  15.460996
          2                  17.483853
          3                   9.372702
          4                   9.827479
          5                  10.522081
          6                  18.083694
          7                   8.123962
          8                   7.562183
          9                  11.454647
2         0                   7.196903
          1                   9.755350
          2                   6.599125
          3                  19.986909
          4                  12.251398
          5                   8.701334
          6                  13.861912
          7                  17.843138
          8                   8.574486
          9                   8.025254

In [102]:
average_df.reset_index(inplace=True)

var_1 = average_df[average_df['system_id'] == 1]['average_time'].var()
var_2 = average_df[average_df['system_id'] == 2]['average_time'].var()

print(var_1, var_2)

# We see that variances are not equal, so we can't use 2 sample t-test

15.753801230389216 21.3293845771231


In [103]:
#your code here: evaluate whether the difference of average waiting times between the two system designs is statistically significant
from scipy.stats import ttest_ind

sample_1 = average_df[average_df['system_id'] == 1]['average_time']
sample_2 = average_df[average_df['system_id'] == 2]['average_time']

# Perform Welch's t-test
t_stat, p_value = ttest_ind(sample_1, sample_2, equal_var=False)

print("T-statistic:", t_stat)
print("P-value:", p_value)

alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two systems.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two systems.")

T-statistic: 0.5719387581290406
P-value: 0.5745956602907121
Fail to reject the null hypothesis: There is no significant difference between the two systems.


In [117]:
from scipy.stats import t

system_1 = average_df.loc[1]['average_time']
system_2 = average_df.loc[2]['average_time']

# Calculate sample means and variances
mean_1 = system_1.mean()
mean_2 = system_2.mean()
var_1 = system_1.var(ddof=1)
var_2 = system_2.var(ddof=1)
R_1 = system_1.shape[0]
R_2 = system_2.shape[0]

# Calculate the standard error of the difference in means
standard_err = np.sqrt((var_1/R_1) + (var_2/R_2))

# Calculate the degrees of freedom for Welch's t-test
s_1_r_1 = (var_1/R_1)**2
s_2_r_2 = (var_2/R_2)**2
df = ((var_1/R_1) + (var_2/R_2))**2 / ((s_1_r_1/(R_1 - 1)) + (s_2_r_2/(R_2 - 1)))

# Critical value from the t-distribution
alpha = 0.05
t_crit = t.ppf(1 - alpha/2, df)

mean_diff = mean_1 - mean_2

# Confidence interval
ci_lower = mean_diff - t_crit * standard_err
ci_upper = mean_diff + t_crit * standard_err

print(f"Mean of System 1: {mean_1}")
print(f"Mean of System 2: {mean_2}")
print(f"95% Confidence Interval for the difference in means: ({ci_lower}, {ci_upper})")

if ci_lower <= 0 <= ci_upper:
    print("The difference in means is not statistically significant.")
else:
    print("The difference in means is statistically significant.")


Mean of System 1: 12.38096298669257
Mean of System 2: 11.279580803689859
95% Confidence Interval for the difference in means: (-2.950928033115974, 5.153692399121396)
The difference in means is not statistically significant.


Answer the following questions:

- Which system configuration is better? Is the difference in performance statistically significant?

**Answer in this markdown chunk:**

- *We can not do any conclusions about the systems performance, because their differnce is not statisticaly significant. The Welsh t-test p-value is not less than our significance level and the confidence interval include 0.*

## Task 2: Metamodels (CRN)

We'd like to create a metamodel for analyzing the **average waiting time** of System 1. Suppose that for reducing the waiting time, we can:
1. Buy more machines (i.e. increase the number of "counters")
1. Upgrade the current machines and therefore decrease the processing times by a so-called reduction factor. That is, if the reduction factor ($rf$) is 10% that means that we're decreasing the processing times by 10%. The formula is: $pt_{upgrade} = pt * (1 - rf)$
1. A combination of both options.

The idea is to understand how our output metric varies when varying this two design variables, we'll use a simple metamodel to approximate the relationship between our design variables and the average waiting time. Recall that we can use Independent Sampling or Common Random Numbers (CRN) when working with metamodels. In this occasion, we'll implement CRN.

When implementing CRN, is important to note the following:

- It is never enough to simply use the same seed on the random-number generator(s). 
- Each random number used in one model for some purpose should be used for the same purpose in the second model—that is, the use of the random numbers must be synchronized. 
- For example, if the $i$ random number is used to generate the processing time of a machine for the 5th car in model 1, then the $i$ random number should be used for the very same purpose in model 2. 

-----

(If you did Task 1, Task 2.1 and 2.2 are exactly the same as before)

**Task 2.1: Create a stream of random numbers for the interarrival times and processing times. Use the same distributions as before, i.e. $exp(1/0.5)$ and $exp(1/0.3)$, respectively. The size of the stream should be at least 1000. Note that this stream must change whenever we set a new seed but must remain the same across the design variables**

**Task 2.2: Modify the `car_generator_crn` to receive this stream of numbers (both, interarrival and processing times) and use them according the counter for each car. Modify also the parts of your code where you're calling this function.**

After these first 2 steps. You should have succesfully implemented CRN.

**Task 2.3: Within the *for* loop for iterate over the replications, nest two more *for* loops for iterate over the number of machines and the reduction factor. In particular, try with a number of machines from 1 to 4 (all integers) and a reduction factor from 0 (no reduction) to 80% with a stepsize of 10%.**

Note that for every seed, for every value of number of machines and for every value of reduction factor, you should run your simulation model. Don't forget to adjust the stream of processing times by the reduction factor and to set the corresponding number of machines when defining the Carwash object.

**Task 2.4: Modify the `EventLogger_Task2` to set the value of the current number of machines and reduction factor (you can implement this with a new method similar to `set_replication_info`). Include this information everytime you log an event (i.e. in every method that starts with `log_`)**

**Task 2.5: Create the "replication table". That is: a table that contains for every replication (<u>and now, for every combination of the design variables</u>) the average waiting time within replications. This should be very similar to Task 1 of Assignment 6 but considering that now we have two design variables.**

**Task 2.6: Aggregate the "replication table" accross replications. That is, you should have the average (across replications) of the average (within replications) waiting time for every combination of our design variables.**

**Task 2.7: Fit a (multiple) linear regression using the average (of the average) waiting time as dependent variable and our two design variables as independent variables. Note that the number of machines is a categorical variable and the reduction factor is a numerical variable. You can use any package for this, check Hint 1 for a suggestion.** 

**Task 2.8: Interpret the results of the (multiple) linear regression and answer the questions.**

-----

**Hint 1:** An easy way of fitting linear regression models in python is to use the `ols` function of the `statsmodels.formula.api` module. Once you have your fitted model, you only have to use the `.summary()` method on the model to get an overview of the regression results. Check the [reference](https://www.statsmodels.org/dev/example_formulas.html) for more details. Check as well the use of the `C()` operator for treating categorical variables.

In [54]:
class EventLogger_Task2:
    def __init__(self):
        self.logs = []
        self.replication = None
        self.seed = None
        self.system_id = None 
    
    def set_system_id(self, system_id): #for this task, we're not using this method, therefore system_id will always be None
        self.system_id = system_id

    def set_replication_info(self, replication, seed):
        self.replication = replication
        self.seed = seed

    #your code here: create a method for setting the design variables (number of machines and reduction factor)

    def log_car_arrival(self, entity, time, interarrival_time):
        #your code here: include the design variables in the logs
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_arrival', 'event_key': entity, #simulation info
                          'interarrival_time': interarrival_time
                          })
    
    def log_car_wash_request(self, entity, time, machine_id, processing_time):
        #your code here: include the design variables in the logs
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_wash_request', 'event_key': entity, #simulation info
                          'machine_id': machine_id, 'processing_time': processing_time
                          })

    def log_car_departure(self, entity, time):
        #your code here: include the design variables in the logs
        self.logs.append({'system_id': self.system_id, #system info
                          'replication_id': self.replication, 'seed': self.seed, #replication info
                          'event_time': time, 'event_name': 'car_departure', 'event_key': entity #simulation info
                          })

    def get_logs_df(self):
        return pd.DataFrame(self.logs)
    
    def dump_logs_df(self, filepath=None):
        if filepath is None: 
            filepath = "logs.csv"

        self.get_logs_df().to_csv(filepath, index=False)

In [55]:
#your code here: modify the function to receive the streams as inputs
def car_generator_crn(env, carwash, logger):

    car_count = 0
    
    # Create cars while the simulation is running
    while True:
        iat = #your code here: use the corresponding stream according the counter for each car
        pt = #your code here: use the corresponding stream the counter for each car

        yield env.timeout(iat) #common random numbers

        Car_S1(env, f'Car {car_count}', carwash, logger, iat, pt)
        car_count += 1

SyntaxError: invalid syntax (3520933659.py, line 8)

In [ ]:
# parameters
SIM_TIME = 8*60    # Simulation time in minutes
N_REPLICATIONS = 10 # Number of Replications

# Setup and start the simulation
print('Running Simulation...')

#define logger (same for all replications)
logger_t2 = EventLogger_Task2()

#Compute a pool of seeds that is larger than the number of replications
safe_factor = 10
pool_of_seeds = range(1, N_REPLICATIONS * safe_factor)

#get a list of seeds of length: N_REPLICATIONS from a pool of seeds. 
#We set replace=False to ensure that we don't reuse the same seed twice.
list_of_seeds = np.random.choice(pool_of_seeds, size=N_REPLICATIONS, replace=False)

for i, seed in enumerate(list_of_seeds):
    print(f'Running Replication {i} with seed: {seed} ...')

    #set random seed
    np.random.seed(seed)
    
    #generate streams of inter-arrival times and processing times ("common random numbers")
    stream_iat = #your code here
    stream_pt = #your code here

    #set replication id and random seed
    logger_t2.set_replication_info(i, seed)

    #your code here: iterate over the number of machines and the reduction factor
    for #your code here
        for #your code here

            # adjust the processing time stream by the reduction factor
            stream_pt_adj = #your code here

            #your code here: set the value of the design variables in the logger
            logger_t2

            # Create an environment and start the setup process
            env = simpy.Environment()

            #define resources
            carwash_s1 = Carwash_S1(env, logger_t2) #your code here: set the corresponding number of machines

            #define processes
            #your code here: pass the streams of inter-arrival times and the (adjusted) processing times to the car generator function
            env.process(car_generator_crn(env, carwash_s1, logger_t2))

            # Execute
            env.run(until=SIM_TIME)

print('... Done')

In [ ]:
#Leave this code unchanged
events_df_t2 = logger_t2.get_logs_df()

events_df_t2

In [ ]:
#Leave this code unchanged (check that the interarrival times are the same for the same car for every combination of the design variables)
boolean_mask = (events_df_t2['replication_id'] == 0) & (events_df_t2['event_key'].isin(['Car 0']) ) & (events_df_t2['event_name'] == 'car_arrival')

events_df_t2[boolean_mask].sort_values(by=['event_key', 'event_name', 'n_machines', 'reduction_factor']).head(10)

In [ ]:
#Leave this code unchanged (check that the processing times are the adjusted for the same car for every combination of the design variables)

boolean_mask = (events_df_t2['replication_id'] == 0) & (events_df_t2['event_key'].isin(['Car 0']) ) & (events_df_t2['event_name'] == 'car_wash_request')

events_df_t2[boolean_mask].sort_values(by=['event_key', 'event_name', 'n_machines', 'reduction_factor']).head(10)

In [ ]:
#your code here: compute the waiting time for each car in each replication and each combination of the design variables (you may want to use pd.pivot_table for this)


In [ ]:
#your code here: compute the average waiting time for each replication and each combination of the design variables (average waiting time within replications, for each combination of the design variables)


In [ ]:
# Aggregate accross replications. 
# your code here: compute the average (of the average) waiting time for each combination of the design variables (average waiting time across replications, for each combination of the design variables)


In [ ]:
#your code here: fit a linear regression model to the data and interpret the results


Answer the following questions:

- What can you say about the "goodnes-of-fit" of the model?
- Assuming you have only 1 machine, what would be better for reducing the average waiting time? (assume both options have the same cost)
    1. Buying one more machine 
    1. Upgrading the machine and reducing the processing time by 50%?. 
- How is the effect of buying more machines? Is this effect linear with the number of machines??

**Answer in this markdown chunk:**

- *Your answer here*